In [ ]:
from pyspark.sql import SparkSession, Row
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType, LongType, DateType, TimestampType
from pyspark.sql.functions import upper, pandas_udf, expr
import pyspark.pandas as ps

from datetime import datetime, date
import pandas as pd
import numpy as np
import os

os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"

In [13]:
spark = (
    SparkSession.builder
    .appName("PandasToSparkTest")
    .master("local[*]")  # dùng toàn bộ CPU
    .config("spark.driver.bindAddress", "127.0.0.1")
    .config("spark.driver.host", "127.0.0.1")
    .config("spark.python.worker.reuse", "false")   # tránh lỗi worker treo
    .config("spark.network.timeout", "300s")
    .config("spark.executor.heartbeatInterval", "60s")
    .config("spark.sql.execution.arrow.pyspark.enabled", "true")
    .config("spark.sql.ansi.enabled", "false")
    .getOrCreate()
)


In [14]:
psdf = ps.read_csv("archive/T_ONTIME_REPORTING_2021_M2.csv")

psdf.describe()

/home/tmkhoa-1812/miniconda3/envs/pyspark3.12/lib/python3.12/site-packages/pyspark/pandas/utils.py:1037: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `read_csv`, the default index is attached which can cause additional overhead.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


,YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,OP_CARRIER_AIRLINE_ID,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN_CITY_MARKET_ID,ORIGIN_STATE_FIPS,ORIGIN_WAC,DEST_AIRPORT_ID,DEST_AIRPORT_SEQ_ID,DEST_CITY_MARKET_ID,DEST_STATE_FIPS,DEST_WAC,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,DEP_DELAY_NEW,DEP_DEL15,DEP_DELAY_GROUP,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,ARR_DELAY_NEW,ARR_DEL15,ARR_DELAY_GROUP,CANCELLED,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,FLIGHTS,DISTANCE,DISTANCE_GROUP,DIV_AIRPORT_LANDINGS
count,332468.0,332468.0,332468.0,332468.000000,332468.000000,332468.000000,332468.000000,332468.000000,3.324680e+05,332468.000000,332468.000000,332468.000000,332468.000000,3.324680e+05,332468.000000,332468.000000,332468.000000,332468.000000,314224.000000,314208.000000,314208.000000,314208.000000,314208.000000,314104.000000,314104.000000,313930.000000,313930.000000,332468.000000,313930.000000,313388.000000,313388.000000,313388.000000,313388.000000,332468.000000,332468.000000,332468.000000,313388.000000,313388.000000,332468.0,332468.000000,332468.000000,332468.000000
mean,2021.0,1.0,2.0,14.755814,4.044910,20018.063937,2778.170269,12646.767996,1.264681e+06,31742.581656,26.708378,56.875729,12646.622096,1.264666e+06,31742.543715,26.709280,56.874159,1301.284560,1302.974111,5.231248,9.375003,0.118199,-0.308837,17.127961,1329.953474,1487.807632,6.979894,1507.144528,1496.202109,-0.364966,10.419008,0.145484,-0.545870,0.055434,0.001955,142.664542,137.297692,113.201696,1.0,802.318031,3.681560,0.003763
std,0.0,0.0,0.0,8.048546,2.011025,372.044411,1799.362463,1554.775243,1.554771e+05,1367.234959,17.021114,26.024303,1554.741879,1.554738e+05,1367.170331,17.021118,26.024472,445.706719,452.471801,47.581908,46.592529,0.322844,1.855955,10.851868,451.522245,459.149742,5.269912,454.002698,460.736579,50.088158,46.601312,0.352589,2.063849,0.228826,0.044173,68.291257,69.761717,68.348685,0.0,560.021978,2.201322,0.134476
min,2021.0,1.0,2.0,1.000000,1.000000,19393.000000,1.000000,10135.000000,1.013506e+06,30070.000000,1.000000,1.000000,10135.000000,1.013506e+06,30070.000000,1.000000,1.000000,5.000000,1.000000,-59.000000,0.000000,0.000000,-2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,-80.000000,0.000000,0.000000,-2.000000,0.000000,0.000000,21.000000,16.000000,8.000000,1.0,31.000000,1.000000,0.000000
25%,2021.0,1.0,2.0,8.000000,2.000000,19790.000000,1200.000000,11292.000000,1.129202e+06,30559.000000,12.000000,34.000000,11292.000000,1.129202e+06,30559.000000,12.000000,34.000000,925.000000,927.000000,-7.000000,0.000000,0.000000,-1.000000,11.000000,947.000000,1128.000000,4.000000,1136.000000,1134.000000,-18.000000,0.000000,0.000000,-2.000000,0.000000,0.000000,92.000000,86.000000,63.000000,1.0,391.000000,2.000000,0.000000
50%,2021.0,1.0,2.0,15.000000,4.000000,19977.000000,2460.000000,12519.000000,1.251902e+06,31453.000000,26.000000,54.000000,12519.000000,1.251902e+06,31453.000000,26.000000,54.000000,1300.000000,1305.000000,-4.000000,0.000000,0.000000,-1.000000,14.000000,1319.000000,1511.000000,6.000000,1522.000000,1517.000000,-9.000000,0.000000,0.000000,-1.000000,0.000000,0.000000,129.000000,123.000000,98.000000,1.0,677.000000,3.000000,0.000000
75%,2021.0,1.0,2.0,22.000000,6.000000,20378.000000,4367.000000,14082.000000,1.408202e+06,32575.000000,42.000000,82.000000,14082.000000,1.408202e+06,32575.000000,42.000000,82.000000,1646.000000,1651.000000,0.000000,0.000000,0.000000,0.000000,19.000000,1706.000000,1845.000000,8.000000,1856.000000,1852.000000,3.000000,3.000000,0.000000,0.000000,0.000000,0.000000,174.000000,170.000000,145.000000,1.0,1047.000000,5.000000,0.000000
max,2021.0,1.0,2.0,28.000000,7.000000,20452.000000,8786.000000,16869.000000,1.686901e+06,36101.000000,78.000000,93.000000,16869.000000,1.686901e+06,36101.000000,78.000000,93.000000,2359.000000,2400.000000,2791.000000,2791.000000,1.000000,12.000000,256.000000,2400.000000,2400.000000,217.000000,2359.000000,2400.000000,2797.000000,2797.00

In [17]:
psdf.isnull().any(axis=0).sum()
psdf.dtype()

17

In [16]:
psdf.isnull().sum().sum()

/home/tmkhoa-1812/miniconda3/envs/pyspark3.12/lib/python3.12/site-packages/pyspark/pandas/generic.py:1373: FutureWarning: The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  warnings.warn(


300903